# 🚀 FORECAST - Запуск с данными из Google Drive

**Команда Pulsetech**

Этот ноутбук загружает большой датасет с Google Drive и запускает полное обучение.

⏱️ **Время выполнения**: зависит от размера данных

⚙️ **Runtime**: Используйте **CPU** (GPU не нужен!)

---

## 📝 Структура данных:

Ожидается файл `forecast_data.zip` на Google Drive со структурой:
```
forecast_data/
  ├── candles.csv      → train_candles.csv
  ├── news.csv         → train_news.csv
  ├── candles_2.csv    → test_candles.csv (public+private)
  └── news_2.csv       → test_news.csv
```

---


## 0. Настройка Runtime (ВАЖНО!)

**Выберите CPU runtime:**
1. В меню сверху: `Runtime` → `Change runtime type`
2. В поле `Hardware accelerator` выберите **None** (это CPU)
3. Нажмите `Save`

⚠️ **GPU не нужен для этой задачи!**


## 1. Подключение Google Drive


In [ ]:
from google.colab import drive
import os

# Монтируем Google Drive
drive.mount('/content/drive')

print("✅ Google Drive подключен!")
print("\n📂 Путь к вашим файлам: /content/drive/MyDrive/")
print("\n⚠️ Убедитесь что файл forecast_data.zip находится в корне MyDrive")
print("   или укажите правильный путь на следующем шаге")


## 2. Клонирование репозитория


In [ ]:
# Клонируем репозиторий
!git clone https://github.com/sokanaid/finnam-forecast-pulsetech.git
%cd finnam-forecast-pulsetech

print("✅ Репозиторий склонирован!")
print("\n📦 Устанавливаем зависимости...")

# Устанавливаем зависимости
!pip install -q -r requirements.txt

print("✅ Зависимости установлены!")


## 3. Распаковка данных из Google Drive


In [ ]:
import zipfile
import shutil

# Путь к вашему ZIP файлу на Google Drive
# ИЗМЕНИТЕ ПУТЬ если файл находится в другой папке!
zip_path = '/content/drive/MyDrive/forecast_data.zip'

# Проверяем существование файла
if not os.path.exists(zip_path):
    print(f"❌ Файл не найден: {zip_path}")
    print("\n📂 Проверьте путь к файлу!")
    print("   Возможные варианты:")
    print("   - /content/drive/MyDrive/forecast_data.zip")
    print("   - /content/drive/MyDrive/Colab Notebooks/forecast_data.zip")
    print("\n💡 Используйте команду ниже чтобы найти файл:")
    print("   !find /content/drive/MyDrive -name 'forecast_data.zip'")
else:
    print(f"✅ Файл найден: {zip_path}")
    file_size_mb = os.path.getsize(zip_path) / 1024 / 1024
    print(f"📦 Размер: {file_size_mb:.1f} MB")
    
    print("\n📂 Распаковываем данные...")
    
    # Распаковываем в текущую директорию
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('.')
    
    print("✅ Данные распакованы!")
    
    # Показываем структуру
    print("\n📁 Структура распакованных файлов:")
    !ls -lh forecast_data/


## 4. Подготовка данных

Переименовываем файлы под структуру, которую ожидает наш код


In [ ]:
import pandas as pd

# Создаем структуру папок
!mkdir -p data/raw

# Копируем и переименовываем файлы
file_mapping = {
    'forecast_data/candles.csv': 'data/raw/train_candles.csv',
    'forecast_data/news.csv': 'data/raw/train_news.csv',
    'forecast_data/candles_2.csv': 'data/raw/public_test_candles.csv',
    'forecast_data/news_2.csv': 'data/raw/test_news.csv',
}

print("📋 Переименовываем и копируем файлы:\n")

for source, target in file_mapping.items():
    if os.path.exists(source):
        shutil.copy(source, target)
        size_mb = os.path.getsize(target) / 1024 / 1024
        
        # Показываем размер и первые строки
        df = pd.read_csv(target, nrows=2)
        print(f"✅ {source}")
        print(f"   → {target}")
        print(f"   Размер: {size_mb:.1f} MB, Колонки: {list(df.columns)}")
        print()
    else:
        print(f"⚠️ Файл не найден: {source}")
        print()

# Для private_test создаем копию public (если нет отдельного файла)
if not os.path.exists('data/raw/private_test_candles.csv'):
    shutil.copy('data/raw/public_test_candles.csv', 'data/raw/private_test_candles.csv')
    print("📝 Создана копия public_test → private_test")

print("\n✅ Данные готовы!")


## 5. 🚀 Обучение модели

Это может занять от нескольких минут до часа в зависимости от размера данных


In [ ]:
import time

start_time = time.time()

print("="*70)
print("🎓 НАЧАЛО ОБУЧЕНИЯ")
print("="*70)

!python train.py

elapsed = time.time() - start_time
print(f"\n⏱️ Время обучения: {elapsed/60:.2f} минут")

# Проверяем что модели созданы
if os.path.exists('models/forecast_models.pkl'):
    size_mb = os.path.getsize('models/forecast_models.pkl') / 1024 / 1024
    print(f"✅ Модели сохранены: models/forecast_models.pkl ({size_mb:.1f} MB)")
else:
    print("❌ Ошибка: модели не созданы!")


## 6. 🔮 Создание предсказаний


In [ ]:
print("="*70)
print("🔮 СОЗДАНИЕ ПРЕДСКАЗАНИЙ")
print("="*70)

!python predict.py

if os.path.exists('submission.csv'):
    print("\n✅ submission.csv создан!")
else:
    print("\n❌ Ошибка: submission.csv не создан!")


## 7. 📊 Анализ результатов и метрик


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

submission = pd.read_csv('submission.csv')
prob_cols = [f'p{i}' for i in range(1, 21)]

print('=' * 70)
print('📊 ДЕТАЛЬНЫЙ АНАЛИЗ РЕЗУЛЬТАТОВ')
print('=' * 70)

print(f'\n✅ Размер: {len(submission)} строк')
print(f'✅ Уникальных тикеров: {submission["ticker"].nunique()}')
print(f'✅ Тикеры: {list(submission["ticker"].unique())}')

# Общая статистика
all_probs = submission[prob_cols].values.flatten()
print(f'\n📈 Общая статистика вероятностей:')
print(f'   Среднее: {all_probs.mean():.4f}')
print(f'   Std: {all_probs.std():.4f}')
print(f'   Min: {all_probs.min():.4f}')
print(f'   Max: {all_probs.max():.4f}')
print(f'   Медиана: {np.median(all_probs):.4f}')

# Распределение
print(f'\n📊 Распределение вероятностей:')
ranges = [(0, 0.2), (0.2, 0.4), (0.4, 0.6), (0.6, 0.8), (0.8, 1.0)]
for low, high in ranges:
    count = ((all_probs >= low) & (all_probs < high)).sum()
    pct = count / len(all_probs) * 100
    print(f'   [{low:.1f}-{high:.1f}): {pct:5.1f}% ({count:,} значений)')

# Тренд по горизонтам
print(f'\n📉 Средние вероятности по горизонтам:')
horizon_means = []
for i in range(1, 21):
    mean_val = submission[f'p{i}'].mean()
    horizon_means.append(mean_val)
    print(f'   Горизонт {i:2d}: {mean_val:.4f}')

# Тренд
trend_diff = horizon_means[0] - horizon_means[-1]
if trend_diff > 0:
    print(f'\n📉 Тренд: УБЫВАЮЩИЙ (Δ={trend_diff:.4f})')
    print('   ✅ Логично: уверенность снижается с горизонтом')
else:
    print(f'\n📈 Тренд: НЕ убывающий (Δ={trend_diff:.4f})')

print('\n' + '=' * 70)


## 8. 📈 Визуализация


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Гистограмма распределения вероятностей
axes[0, 0].hist(all_probs, bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Распределение вероятностей', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Вероятность')
axes[0, 0].set_ylabel('Количество')
axes[0, 0].axvline(all_probs.mean(), color='red', linestyle='--', label=f'Среднее: {all_probs.mean():.3f}')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 2. Тренд по горизонтам
axes[0, 1].plot(range(1, 21), horizon_means, marker='o', linewidth=2)
axes[0, 1].set_title('Средняя вероятность по горизонтам', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Горизонт (дни)')
axes[0, 1].set_ylabel('Средняя вероятность')
axes[0, 1].grid(alpha=0.3)
axes[0, 1].axhline(0.5, color='gray', linestyle='--', alpha=0.5)

# 3. Boxplot по горизонтам (каждые 4 горизонта)
boxplot_data = [submission[f'p{i}'].values for i in [1, 5, 10, 15, 20]]
axes[1, 0].boxplot(boxplot_data, labels=['1д', '5д', '10д', '15д', '20д'])
axes[1, 0].set_title('Распределение по ключевым горизонтам', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('Вероятность')
axes[1, 0].grid(alpha=0.3)

# 4. Средние вероятности по тикерам
ticker_means = submission.groupby('ticker')[prob_cols].mean().mean(axis=1).sort_values()
axes[1, 1].barh(ticker_means.index, ticker_means.values)
axes[1, 1].set_title('Средняя вероятность по тикерам', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Средняя вероятность')
axes[1, 1].axvline(0.5, color='gray', linestyle='--', alpha=0.5)
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Визуализация готова!")


## 9. 📥 Скачивание результатов


In [ ]:
from google.colab import files

# Скачиваем submission
if os.path.exists('submission.csv'):
    files.download('submission.csv')
    print("✅ submission.csv скачан!")
else:
    print("❌ Файл не найден!")

# Опционально: сохранить в Google Drive
save_to_drive = input("\n💾 Сохранить submission.csv в Google Drive? (y/n): ")
if save_to_drive.lower() == 'y':
    output_path = '/content/drive/MyDrive/submission_forecast.csv'
    shutil.copy('submission.csv', output_path)
    print(f"✅ Сохранено в: {output_path}")

print("\n🎉 Готово! Теперь можете отправить submission.csv на лидерборд")


---

## ✅ Готово!

**Результаты:**
- ✅ Модель обучена на ваших данных
- ✅ Предсказания созданы
- ✅ Метрики проанализированы
- ✅ Визуализация построена
- ✅ submission.csv готов для отправки

**Следующие шаги:**
1. Отправьте `submission.csv` на лидерборд
2. Обновите метрики в презентации реальными значениями
3. Сравните с предыдущими результатами

---

**Команда Pulsetech** 🚀
